# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [ ]:
#!pip install chromadb langchain langchain_community langchain_openai

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [3]:
llm = OpenAI(temperature=0)

In [4]:
from pathlib import Path

doc_path = Path.cwd() / "data" / "sonnets.txt"
print(doc_path)

c:\Users\007T\OneDrive\Desktop\ironhack2025\RAG\data\sonnets.txt


In [ ]:
!pip install chromadb

In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install lxml


In [8]:
import chromadb
import chromadb.config
from langchain_community.vectorstores import Chroma

In [9]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [10]:
sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [11]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [16]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)



Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [17]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [18]:
from langchain.agents import Tool

tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description=(
            "Useful for when you need to answer questions about Shakespeare's Sonnets. "
            "Input should be a fully formed question."
        ),
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description=(
            "Useful for when you need to answer questions about Ruff, the Python linter. "
            "Input should be a fully formed question."
        ),
    ),
]


In [19]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\007T\AppData\Local\Temp\ipykernel_16304\1834837320.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [21]:
agent.invoke(
    "What metaphor does Shakespeare use in Sonnet 18 to compare his beloved to a summer’s day?"
)




> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What metaphor does Shakespeare use in Sonnet 18 to compare his beloved to a summer’s day?"
Observation:  Shakespeare uses the metaphor of a summer's day to describe his beloved's beauty and eternal nature.
Thought: I now know the final answer.
Final Answer: The metaphor used in Sonnet 18 is a summer's day.

> Finished chain.


{'input': 'What metaphor does Shakespeare use in Sonnet 18 to compare his beloved to a summer’s day?',
 'output': "The metaphor used in Sonnet 18 is a summer's day."}

In [22]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between tools before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.
Thought: This information is helpful, but I should also consider the specific needs of my project.
Action: Sonnets QA System
Action Input: "What are some potential benefits of using ruff over flake8?"
Observation:  Some potential benefits of using ruff over flake8 could include improved code readability, better error detection and reporting, and more customizable linting rules. Additionally, ruff may offer

{'input': 'Why use ruff over flake8?',
 'output': 'The potential benefits of using ruff over flake8 include improved code readability, better error detection and reporting, and more customizable linting rules. Additionally, ruff may offer more advanced features and integrations with other tools, making it a more comprehensive solution for code analysis and formatting. Ultimately, the best tool to use will depend on the specific needs and preferences of the project.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [23]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description=(
            "Useful for when you need to answer questions about Shakespeare's Sonnets. "
            "Input should be a fully formed question."
        ),
        return_direct=True
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description=(
            "Useful for when you need to answer questions about Ruff, the Python linter. "
            "Input should be a fully formed question."
        ),
        return_direct=True
    ),
]

In [24]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [26]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question since it is about a political event.
Action: Ruff QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the State of the Union address?"
Observation:  I don't know.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " I don't know."}

In [27]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 It's important to understand the differences between tools before deciding which one to use.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': " Ruff implements more rules than Flake8 and does not support custom lint rules. It also has some minor differences in its rule set and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations, while Flake8 supports plugins and allows for custom and third-party rules."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [28]:
tools = [
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description=(
            "Useful for when you need to answer questions about Shakespeare's Sonnets. "
            "Input should be a fully formed question."
        ),
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description=(
            "Useful for when you need to answer questions about Ruff, the Python linter. "
            "Input should be a fully formed question."
        ),
    ),
]

In [29]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [30]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: Did the president mention that tool in the state of the union?
Observation:  I don't know.
Thought: I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: Did the president mention that tool in the state of the union?
Observation:  I don't know.
Thought: I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  I don't know.
Thought: I should use the Ruff QA System to answer this question.
Action: Ruff QA System
Action Input: What tool

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.'}